In [5]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib

%cd /root/text_to_mesh

from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-kfbenep6
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-kfbenep6
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[Errno 2] No such file or directory: '/root/text_to_mesh'
/root/text-to-mesh


In [6]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,    
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")      

pkg = torch.load("./MeshGPT-autoencoder_2024-06-11T14_04_37.942897+00_00.pt") 
autoencoder.load_state_dict(pkg['model'])
for param in autoencoder.parameters():
    param.requires_grad = True

# transformer.conditioner.text_models[0].model.to("cuda")
# transformer = transformer.to("cuda")
# dataset.embed_texts(transformer,1)
 
dataset = MeshDataset.load("./labels_885_10x5_21720_mod.npz") 
labels = list(set(item['texts'] for item in dataset.data))
dataset.generate_codes(autoencoder,150)
dataset.data[0].keys() 

transformer = MeshTransformer(
    autoencoder,
    dim = 768,
    coarse_pre_gateloop_depth =2,  
    fine_pre_gateloop_depth= 2, 
    attn_depth = 12,  
    attn_heads = 12, 
    fine_cross_attend_text = True,
    text_cond_with_film = False,
    cross_attn_num_mem_kv = 4,
    num_sos_tokens = 1, 
    dropout  = 0.0,
    max_seq_len = 1500, 
    fine_attn_depth = 2,
    condition_on_text = True, 
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", 
    text_condition_cond_drop_prob = 0.25, 
).to("cuda")

dataset.embed_texts(transformer,1)

batch_size = 2
grad_accum_every =4     
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                  accelerator_kwargs = {"mixed_precision" : "fp16"}, optimizer_kwargs = { "eps": 1e-7} , 
                                 learning_rate = 1e-3, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(35, stop_at_loss = 0.00005)   
 
# pkg = torch.load("./MeshGPT-transformer_trained_base.pt") 
# transformer.load_state_dict(pkg['model'],strict=False)

batch_size =16
grad_accum_every =4      
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                 learning_rate = 1e-4, batch_size=batch_size, checkpoint_every_epoch = 25) 

total_epochs = 740
epochs_per_save = 25

for i in range(epochs_per_save, total_epochs + 1, epochs_per_save):
    loss = trainer.train(i, stop_at_loss = 0.00005)   
    pkg = dict( model = transformer.state_dict(), ) 
    now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat().replace(":", "_")
    torch.save(pkg, str(f"./MeshGPT-transformer_trained_{now_utc}.pt"))

[MeshDataset] Loaded 15170 entrys
[MeshDataset] Created from 15170 entrys


100%|██████████| 102/102 [00:06<00:00, 16.37it/s]


[MeshDataset] Generated codes for 15170 entrys


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[MeshDataset] Generated 885 text_embeddings


Epoch 1/35:   0%|          | 0/7585 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
Epoch 1/35: 100%|██████████| 7585/7585 [06:15<00:00, 20.21it/s, loss=nan] 


Epoch 1 average loss: nan


Epoch 2/35:  18%|█▊        | 1385/7585 [01:06<04:58, 20.74it/s, loss=nan]


KeyboardInterrupt: 

In [ ]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,   
        codebook_size =  2048, 
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

torch.cuda.empty_cache()
gc.collect()  
 
transformer = MeshTransformer(
    autoencoder,
    dim = 768,
    coarse_pre_gateloop_depth =2,  
    fine_pre_gateloop_depth= 2, 
    attn_depth = 12,  
    attn_heads = 12, 
    fine_cross_attend_text = True,
    text_cond_with_film = False,
    cross_attn_num_mem_kv = 4,
    num_sos_tokens = 1, 
    dropout  = 0.0,
    max_seq_len = 1500, 
    fine_attn_depth = 2,
    condition_on_text = True, 
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", 
    text_condition_cond_drop_prob = 0.25, 
).to("cuda")

pkg = torch.load("./MeshGPT-transformer_trained_base.pt") 
transformer.load_state_dict(pkg['model'],strict=False)
 
dataset = MeshDataset.load("./labels_885_10x5_21720_mod.npz") 
labels = list(set(item['texts'] for item in dataset.data))
dataset.generate_codes(autoencoder,150)
dataset.data[0].keys() 
dataset.embed_texts(transformer,1)

batch_size =16
grad_accum_every =4      
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                 learning_rate = 1e-4, batch_size=batch_size, checkpoint_every_epoch = 25) 

total_epochs = 740
epochs_per_save = 25

for i in range(epochs_per_save, total_epochs + 1, epochs_per_save):
    loss = trainer.train(i, stop_at_loss = 0.00005)   
    pkg = dict( model = transformer.state_dict(), ) 
    now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat().replace(":", "_") 
    torch.save(pkg, str(f"./MeshGPT-transformer_trained_{now_utc}.pt"))